Импортируем библиотеки для работы: numpy, pandas, а также matplotlib и seaborn для визуализации данных.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
from IPython.core.pylabtools import figsize
figsize(13, 10)

Откроем файл с Датасетом и просмотрим информацию о данной таблице.

In [2]:
data = pd.read_csv('/home/akimg/Downloads/heart.csv', sep=',')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
age         303 non-null int64
sex         303 non-null int64
cp          303 non-null int64
trestbps    303 non-null int64
chol        303 non-null int64
fbs         303 non-null int64
restecg     303 non-null int64
thalach     303 non-null int64
exang       303 non-null int64
oldpeak     303 non-null float64
slope       303 non-null int64
ca          303 non-null int64
thal        303 non-null int64
target      303 non-null int64
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


Разделим данные на тренировочную и валидационную выборки.

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis='columns'),
                                                    data['target'],
                                                    stratify=data['target'])

создадим функцию, которая удаляет определенные колонки из данной таблицы.

In [4]:
def columns_dropper(X):
    X = X.drop(['sex', 'thalach', 'exang', 'oldpeak', 'ca'], axis='columns')
    return X

In [5]:
columns_dropper(X_train).columns

Index(['age', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'slope', 'thal'], dtype='object')

Создадим еще одну функцию, которая будет преобразовывать типы указанных колонок в таблице.

In [6]:
from collections import Counter
counter = Counter(X_train['thal'])
cols = ['age', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'slope', 'thal']

def datatype_transform(X):
    for col in cols:
        X[col] = X[col].apply(lambda value: 0.5 if pd.isnull(value) else value)
    X.loc[:, 'thal'] = X['thal'].apply(lambda title: counter.get(title, 0))
    return X

Создадим класс, который сделает из обычной функции, принимающей матрицу признаков, шаг пайплайна, также создадим класс, который нормализует признаки.

In [7]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import FunctionTransformer

pipeline = make_pipeline(
    FunctionTransformer(columns_dropper, validate=False),

    FunctionTransformer(datatype_transform, validate=False),
    MinMaxScaler(),
    KNeighborsClassifier(n_neighbors=5),
)

pipeline

Pipeline(memory=None,
         steps=[('functiontransformer-1',
                 FunctionTransformer(accept_sparse=False, check_inverse=True,
                                     func=<function columns_dropper at 0x7fff94e8fb70>,
                                     inv_kw_args=None, inverse_func=None,
                                     kw_args=None, pass_y='deprecated',
                                     validate=False)),
                ('functiontransformer-2',
                 FunctionTransformer(accept_sparse=False, check_inverse=True,
                                     func=<function datatype_transform at 0x7fffbc7f8f28>,
                                     inv_kw_args=None, inverse_func=None,
                                     kw_args=None, pass_y='deprecated',
                                     validate=False)),
                ('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('kneighborsclassifier',
                 KNeighborsClassifi

In [8]:
pipeline.fit(X_train, y_train)
pipeline.score(X_test, y_test)

0.8421052631578947

# Сохраним модель

Импортируем библиотеку pickle для дампа модели. Далее откроем файл для записи данных.

In [9]:
import pickle

with open("/home/akimg/Downloads/lab2_model.pickle", 'wb') as opened_file:
    pickle.dump(pipeline, opened_file)

Проверим рузультат дампа:

In [10]:
import pandas as pd
import pickle

with open("/home/akimg/Downloads/lab2_model.pickle", 'rb') as f:
    model = pickle.load(f)
    
model

Pipeline(memory=None,
         steps=[('functiontransformer-1',
                 FunctionTransformer(accept_sparse=False, check_inverse=True,
                                     func=<function columns_dropper at 0x7fff94e8fb70>,
                                     inv_kw_args=None, inverse_func=None,
                                     kw_args=None, pass_y='deprecated',
                                     validate=False)),
                ('functiontransformer-2',
                 FunctionTransformer(accept_sparse=False, check_inverse=True,
                                     func=<function datatype_transform at 0x7fffbc7f8f28>,
                                     inv_kw_args=None, inverse_func=None,
                                     kw_args=None, pass_y='deprecated',
                                     validate=False)),
                ('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('kneighborsclassifier',
                 KNeighborsClassifi

Заново считаем данные.

In [12]:
data = pd.read_csv('/home/akimg/Downloads/heart.csv', sep=',')
X = data.drop('target', axis='columns')
y = data['target']

model.score(X, y)

0.834983498349835

# Сохраним модель первой лабы